In [2]:
import requests
import pandas as pd
import numpy as np
import time
import pprint as pp
from tqdm import tqdm
import time
import psutil
import sys

#Collect zip codes and categories
zip_codes = [line.rstrip('\n') for line in open('zips.txt')]
categories = [line.rstrip('\n') for line in open('categories.txt')]
southern_zips = [line.rstrip('\n') for line in open('southern_zips.txt')]
#zip_codes_remaining = [line.rstrip('\n') for line in open('zip_codes_remaining.txt')]

#Define the API Key, Endpoint, and Header
API1='ds5abV86lpgzBp767VpgjncvxHLDI64ZqZQABokWL-sRtu6II83zKSLuvhxZNEaHJ_tJ5aUFJRdlIMGnudVKQv61YUkS_vq8AuGeJOz9oPGfyvELw3rDVAVyUWdpXXYx'
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {'Authorization': 'bearer %s' % API3}

#Create a copy of zip_codes
zip_codes_remaining = zip_codes.copy()

#Initialize lists for collecting data
name = []
url = []
phone = []
address = []
city = []
state = []
czip = []
rating = []
review_count = []
vertical = []
search = []
yelp_id = []
review1 = []
review2 = []
review3 = []



In [3]:
HEADERS = {'Authorization': 'bearer %s' % API1}

In [3]:
#zip_codes_remaining = zip_codes.copy()

In [4]:
len(zip_codes_remaining)

38933

In [5]:
# #Create Zip Codes Remaining
# with open('zip_codes_remaining.txt','w') as f:
#     for x in zip_codes_remaining:
#         f.write(str(x)+'\n')
# f.close()

In [4]:
PARAMETERS = {'location':'92122', 
                          'term': 'liquor store',
                          'limit': 50,
                          'offset':50}

response = requests.get(url = ENDPOINT, params = PARAMETERS, headers = HEADERS)

In [6]:
business_data = response.json()

In [8]:
response_reviews = requests.get(url = 'https://api.yelp.com/v3/businesses/_D8Z-If-lsmaFOMtP-6Z3Q/reviews',
                               params = {}, headers = HEADERS)

In [9]:
review_data = response_reviews.json()

In [13]:
review_data['reviews'][0]['text']

'Rob was kind enough to deliver the order himself during a global pandemic. He also followed social distancing rules by leaving the order right at my door...'

In [6]:
categories = ['liquor store']

In [8]:
count=0
for zips in tqdm(np.random.choice(zip_codes_remaining,10)):
    try:
        zip_codes_remaining.remove(zips)
    except:
        print(zips)
    #Check if zip code is in a southern state
    for cat in categories:
        count+=1
        print(count)
        try:
            PARAMETERS = {'location':str(zips), 
                      'term': cat,
                      'limit': 50,
                      'offset':50}
            response = requests.get(url = ENDPOINT, params = PARAMETERS, headers = HEADERS)
            if response.status_code!=200:
                print('Error: response status code not 200')
                break
            business_data = response.json()
            for business in business_data['businesses']:
                name = name + [business['name']]
                url = url + [business['url']]
                address = address + [business['location']['address1']]
                city = city + [business['location']['city']]
                state = state + [business['location']['state']]
                czip = czip + [business['location']['zip_code']]
                phone = phone + [business['phone']]
                search = search + [cat]
                yelp_id = yelp_id + [business['id']]
                industries = str()
                for i in business['categories']:
                    industries = industries+', '+str(i['title'])
                vertical = vertical + [industries[2:]]
                rating = rating + [business['rating']]
                review_count = review_count + [business['review_count']]
        except:
            pass

  0%|          | 0/10 [00:00<?, ?it/s]

1


 10%|█         | 1/10 [00:00<00:05,  1.57it/s]

2


 20%|██        | 2/10 [00:01<00:04,  1.73it/s]

...

 80%|████████  | 8/10 [00:06<00:01,  1.10it/s]

9


 90%|█████████ | 9/10 [00:06<00:00,  1.27it/s]

10


100%|██████████| 10/10 [00:07<00:00,  1.43it/s]


In [9]:
len(zip_codes_remaining)

38922

In [10]:
result = pd.DataFrame({
    'Name':name,
    'URL':url,
    'Phone':phone,
    'Address':address,
    'City':city,
    'State':state,
    'ZIP':czip,
    'Vertical':vertical,
    'Rating':rating,
    'Review Count':review_count,
    'Search':search,
    'YelpID':yelp_id
})
print('Original length of collected data: '+str(len(result))+' leads.')

Original length of collected data: 179 leads.


In [15]:
#Create dataframe to aggregate all of the data collected
result = pd.DataFrame({
    'Name':name,
    'URL':url,
    'Phone':phone,
    'Address':address,
    'City':city,
    'State':state,
    'ZIP':czip,
    'Vertical':vertical,
    'Rating':rating,
    'Review Count':review_count,
    'Search':search,
    'YelpID':yelp_id
})
print('Original length of collected data: '+str(len(result))+' leads.')

#Reformat phone numbers
result['Phone'] = result['Phone'].apply(lambda x: x[2:])

result = result[result['Review Count']<10]

#Remove a few unique outliers
result = result.drop(index = result[result['Name']=='Massey Services'].index)
result = result.drop(index = result[result['ZIP']=='BC'].index)
result = result.drop(index = result[result['ZIP']=='XWY'].index)

#Drop duplicates
result = result.drop_duplicates(subset=['Name'], keep='first')

already_called = pd.read_csv('ytel.csv')
already_called = list(already_called['phone_number_dialed'].apply(lambda x: "{:f}".format(float(x))[:10]))
result = result[-result['Phone'].isin(already_called)]

Original length of collected data: 230905 leads.


In [16]:
already_called_campaign = pd.read_csv('3rdPage.csv')
already_called_campaign = list(already_called_campaign['Name'])
result = result[-result['Name'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('4thPage.csv')
already_called_campaign = list(already_called_campaign['Name'])
result = result[-result['Name'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('GirthyGroup.csv')
already_called_campaign = list(already_called_campaign['Name'])
result = result[-result['Name'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('Spooktober.csv')
already_called_campaign = list(already_called_campaign['Name'])
result = result[-result['Name'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('ML2-0.csv')
already_called_campaign = list(already_called_campaign['Name'])
result = result[-result['Name'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('ML2-1.csv')
already_called_campaign = list(already_called_campaign['Name'])
result = result[-result['Name'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('yelp_ml_v2.csv')
already_called_campaign = list(already_called_campaign['Name'])
result = result[-result['Name'].isin(already_called_campaign)]

try:
    already_called_campaign = pd.read_csv('phone_burner_02-03-2020.csv')
    already_called_campaign = list(already_called_campaign['Name'])
    result = result[-result['Name'].isin(already_called_campaign)]
except: pass

try:
    already_called_campaign = pd.read_csv('phone_burner_02-05-2020.csv')
    already_called_campaign = list(already_called_campaign['Name'])
    result = result[-result['Name'].isin(already_called_campaign)]
except: pass

try:
    already_called_campaign = pd.read_csv('phone_burner_02-05-2020_2.csv')
    already_called_campaign = list(already_called_campaign['Name'])
    result = result[-result['Name'].isin(already_called_campaign)]
except: pass

try:
    already_called_campaign = pd.read_csv('phone_burner_02-07-2020.csv')
    already_called_campaign = list(already_called_campaign['Name'])
    result = result[-result['Name'].isin(already_called_campaign)]
except: pass

try:
    already_called_campaign = pd.read_csv('phone_burner_02-10-2020.csv')
    already_called_campaign = list(already_called_campaign['Name'])
    result = result[-result['Name'].isin(already_called_campaign)]
except: pass

In [17]:
len(result)

64134

In [18]:
result.to_csv('result.csv')

In [19]:
def yelp_to_yp(x):
    if x == "Contractors": return "Home Improvement & Remodeling"
    elif x == "Movers": return "Locksmith"
    elif x == "Electricians": return "Electrician"
    elif x == "Air Conditioner & Heating": return "Heating & Air"
    elif x == "Home Cleaners": return "Cleaning"
    elif x == "Painters": return "Painting"
    elif x == "Landscapers": return "Landscaping & Trees"
    elif x == "Movers": return "Moving & Storage"
    else: return x

In [24]:
result.Search = result.Search.apply(yelp_to_yp)

In [25]:
result

,Name,URL,Phone,Address,City,State,ZIP,Vertical,Rating,Review Count,Search,YelpID
3,JB Complete Plumbing Services,https://www.yelp.com/biz/jb-complete-plumbing-...,8166280655,6609 Royal St,Pleasant Valley,MO,64068,"Plumbing, Water Heater Installation/Repair, Co...",4.5,2,Home Improvement & Remodeling,Gd-dJxeCNI5kDxEk-UHY1w
39,Aztec House Leveling,https://www.yelp.com/biz/aztec-house-leveling-...,2105998222,9214 Converse Business Ln,Converse,TX,78109,"Contractors, Roofing, Plumbing",5.0,1,Home Improvement & Remodeling,KqCpRM2i0d5gperHFCFBdw
45,Cabinet Depot,https://www.yelp.com/biz/cabinet-depot-san-ant...,2107217780,2620 Blanco Rd,San Antonio,TX,78212,"Cabinetry, Contractors, Building Supplies",5.0,1,Home Improvement & Remodeling,q20eM-O0Tc2lGu7QNRIxdw
46,Texas Remodeling Pros,https://www.yelp.com/biz/texas-remodeling-pros...,2107742666,2550 Boardwalk St,San Antonio,TX,78217,"Patio Coverings, Painters, Contractors",5.0,3,Home Improvement & Remodeling,2lr8MxFkgcC0tCMP5YJ6Gg
53,Quik Concrete Solutions,https://www.yelp.com/biz/quik-concrete-solutio...,5122106522,700 Engelke Rd,Niederwald,TX,78640,"Contractors, Masonry/Concrete",5.0,1,Home Improvement & Remodeling,PoGQ_yzfYfQsSnC4ICvMxw
59,Diamante Custom Homes,https://www.yelp.com/biz/diamante-custom-homes...,2103416430,4725 College Park,San Antonio,TX,78249,Contractors,5.0,1,Home Improvement & Remodeling,LDGyAfUhyQTC3J4gYodYgg
60,Calvin Weed Excavation,https://www.yelp.com/biz/calvin-weed-excavatio...,5128479001,700 Ledgerock Rd,Wimberley,TX,78676,Contractors,5.0,1,Home Improvement & Remodeling,C2egd5jicfwCqczhLUWKjA
71,B&B Masonry,https://www.yelp.com/biz/b-and-b-masonry-san-a...,2102873435,None,San Antonio,TX,78221,"Masonry/Concrete, Stucco Services, Contractors",5.0,1,Home Improvement & Remodeling,o0ZFyInmN7g0AHTzHI9iCg
95,Swan Building & Design,https://www.yelp.com/biz/swan-building-and-des...,5105955337,86 Hillcrest Rd,Berkeley,CA,94705,Contractors,4.0,5,Home Improvement & Remodeling,d6v9exDcbJOy8DwuP2-48w
96,Carpentry and Such,https://www.yelp.com/biz/carpentry-and-such-el...,,7249 Blake St,El Cerrito,CA,94530,Contractors,5.0,1,Home Improvement & Remodeling,03mNLDqyCqxN4ZL-jW3DQw


In [26]:
result[result['Search']=='Mobile Detailer'].to_csv('jeff.csv')

In [18]:
result.to_csv('temp_yelp.csv')

In [ ]:
#DO NOT RUN

In [20]:
result2 = result.copy()

In [14]:
result1 = result.copy()

In [5]:
df = pd.read_csv('yelp.csv')

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
df['Phone'] = df['Telephones'].apply(lambda x: x.split(';')[0].split('+')[1].replace('-','')[1:] if len(x)>10 else float('NaN'))

In [15]:
df.drop('Telephones', axis=1, inplace=True)

In [17]:
df.columns

Index(['Domain', 'Location on Site', 'Tech Spend USD', 'Company', 'Vertical',
       'Quantcast', 'Alexa', 'Majestic', 'Umbrella', 'Emails', 'Twitter',
       'Facebook', 'LinkedIn', 'Google', 'Pinterest', 'GitHub', 'Instagram',
       'Vk', 'Vimeo', 'Youtube', 'People', 'City', 'State', 'Zip', 'Country',
       'First Detected', 'Last Found', 'First Indexed', 'Last Indexed',
       'Exclusion', 'GDPR', 'Phone'],
      dtype='object')

In [19]:
df = df[['Company','Domain','Tech Spend USD','Vertical','City','State','Zip','Phone']]

In [26]:
df.to_csv('builtwith_yelp.csv')

In [30]:
print('ENDED UP WITH: '+str(len(result))+' LEADS')

ENDED UP WITH: 18154 LEADS


In [33]:
result.reset_index().drop('index',axis=1).to_csv('yelp_ml_v2.csv')

In [39]:
ML2 = pd.read_csv('ML2-1.csv')

In [40]:
pd.concat([ML2.drop('Unnamed: 0', axis=1), result])

,Name,URL,Phone,Address,City,State,ZIP,Vertical,Rating,Review Count,Search,YelpID
0,American Dream Heating and Cooling,https://www.yelp.com/biz/american-dream-heatin...,6.3077e+09,246 Butternut Dr,Bolingbrook,IL,60440,"Heating & Air Conditioning/HVAC, Electricians",1.0,1,Electricians,6IcM5nlZM5RuRvsZevsCIQ
1,Cassandra's Cleaning,https://www.yelp.com/biz/cassandras-cleaning-w...,6.30563e+09,NaN,Wheaton,IL,60187,Home Cleaning,5.0,1,Home Cleaners,EMyxQ69bxy5S7PeDF4ssMg
2,M & M Service Plumbing,https://www.yelp.com/biz/m-and-m-service-plumb...,7.0845e+09,3010 W Division,Melrose Park,IL,60160,"Plumbing, Water Heater Installation/Repair",3.0,2,Plumbers,HjIjqdzwhxGN_iXNo18eKw
3,Colorado-Designs,https://www.yelp.com/biz/colorado-designs-west...,3.03905e+09,11411 West 107th Pl,Westminster,CO,80021,"Contractors, Carpenters, Kitchen & Bath",5.0,1,Contractors,z33V4oAPwG-SN9A9z5J71A
4,Trophy Outdoor Environments,https://www.yelp.com/biz/trophy-outdoor-enviro...,8.17235e+09,NaN,Mansfield,TX,76063,"Landscaping, Pool & Hot Tub Service",1.0,1,Landscapers,qxU_prYYUKPgsg2zXEEsMA
5,M-Co Electric,https://www.yelp.com/biz/m-co-electric-el-mont...,8.00619e+09,NaN,El Monte,CA,91732,"Electricians, Lighting Fixtures & Equipment, G...",5.0,1,Electricians,taYRQxgWDP0XIDF4NKJoAg
6,In Place,https://www.yelp.com/biz/in-place-piedmont-2?a...,8.77578e+09,NaN,Piedmont,CA,94611,"Movers, Home Organization, Packing Services",5.0,4,Movers,sgiC7s-rjJSRoRlB3qsL6A
7,Economove,https://www.yelp.com/biz/economove-richmond-2?...,5.10541e+09,815 Harbour Way S,Richmond,CA,94804,Movers,5.0,4,Movers,oCzVwYxZS34x-AMl0kGeFA
8,North Star Contractors,https://www.yelp.com/biz/north-star-contractor...,9.5498e+09,1180 SW 36th Ave,Pompano Beach,FL,33069,Contractors,3.0,2,Contractors,Tu9km0jFudKVuZrCR84OEg
9,Crognale Construction,https://www.yelp.com/biz/crognale-construction...,9.54771e+09,2801 Ne 51st St,Fort Lauderdale,FL,33308,Contractors,1.0,1,Contractors,qVuz18jBp3VL8L49iuB7yA


In [ ]:
# Cross reference phone number

In [29]:
no_website = pd.read_csv('no_website.csv')
builtwith_yelp = pd.read_csv('builtwith_yelp.csv')

In [24]:
builtwith_yelp['Phone'] = builtwith_yelp['Phone'].apply(lambda x: "{:f}".format(float(x))[:10])

In [32]:
result = no_website

In [33]:
already_called = pd.read_csv('ytel.csv')
already_called = list(already_called['phone_number_dialed'].apply(lambda x: "{:f}".format(float(x))[:10]))
result = result[-result['Phone'].isin(already_called)]

already_called_campaign = pd.read_csv('3rdPage.csv')
already_called_campaign = list(already_called_campaign['Phone'].apply(lambda x: "{:f}".format(float(x))[:10]))
result = result[-result['Phone'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('4thPage.csv')
already_called_campaign = list(already_called_campaign['Phone'].apply(lambda x: "{:f}".format(float(x))[:10]))
result = result[-result['Phone'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('GirthyGroup.csv')
already_called_campaign = list(already_called_campaign['Phone'].apply(lambda x: "{:f}".format(float(x))[:10]))
result = result[-result['Phone'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('Spooktober.csv')
already_called_campaign = list(already_called_campaign['Phone'].apply(lambda x: "{:f}".format(float(x))[:10]))
result = result[-result['Phone'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('ML2-0.csv')
already_called_campaign = list(already_called_campaign['Phone'].apply(lambda x: "{:f}".format(float(x))[:10]))
result = result[-result['Phone'].isin(already_called_campaign)]

already_called_campaign = pd.read_csv('ML2-1.csv')
already_called_campaign = list(already_called_campaign['Phone'].apply(lambda x: "{:f}".format(float(x))[:10]))
result = result[-result['Phone'].isin(already_called_campaign)]

In [31]:
len(no_website)

10307

In [35]:
result.to_csv('no_website_filtered.csv')

In [2]:
df = pd.read_csv('yelp_ml_v2.csv')

In [3]:
df

,Unnamed: 0,Name,URL,Phone,Address,City,State,ZIP,Vertical,Rating,Review Count,Search,YelpID
0,0,Infinity Remodeling,https://www.yelp.com/biz/infinity-remodeling-l...,8.008530e+09,NaN,Los Angeles,CA,90035,Contractors,5.0,2,Home Improvement & Remodeling,9vDNwZxVHVCd3ERcPzyNow
1,1,Lavie construction,https://www.yelp.com/biz/lavie-construction-lo...,8.446472e+09,16200 Ventura Blvd,Los Angeles,CA,91436,Contractors,3.5,9,Home Improvement & Remodeling,7xRJs2DIzjaSyNeEja3W3g
2,2,Reza's Custom Tile Work,https://www.yelp.com/biz/rezas-custom-tile-wor...,3.107489e+09,2224 Walker Ave,San Pedro,CA,90731,"Tiling, Grout Services, Patio Coverings",5.0,8,Home Improvement & Remodeling,l7wBsmXLvVgQ3RYjAt4dRQ
3,3,All Shades Of Green,https://www.yelp.com/biz/all-shades-of-green-l...,6.262759e+09,2120 N Hillhurst Ave,Los Angeles,CA,90027,"Windows Installation, Contractors, Solar Insta...",5.0,1,Home Improvement & Remodeling,Tvl_oSGyOewIpPh8ZPBbCg
4,4,Watson Construction,https://www.yelp.com/biz/watson-construction-w...,5.624849e+09,NaN,Whittier,CA,90808,Contractors,3.0,2,Home Improvement & Remodeling,ubogOrIEwRlVuJKmhJRNvg
5,5,Potter Home Improvement Services,https://www.yelp.com/biz/potter-home-improveme...,5.625370e+09,NaN,Long Beach,CA,90807,"Handyman, Roofing, Windows Installation",3.5,6,Home Improvement & Remodeling,w-GH5J6o2bTATbqsg5Hg_A
6,6,Ban A Pest,https://www.yelp.com/biz/ban-a-pest-inglewood-...,3.103793e+09,NaN,Inglewood,CA,90301,Pest Control,5.0,2,Pest Control,bLxnFFFZiLjfT_m0_KmdHg
7,7,Travis Termite,https://www.yelp.com/biz/travis-termite-torran...,3.105299e+09,NaN,Torrance,CA,90501,Pest Control,3.5,3,Pest Control,HHh9AAcD-aZGD5iJt5Ho9Q
8,8,Culver City Pest Control,https://www.yelp.com/biz/culver-city-pest-cont...,3.103978e+09,12120 W Washington Blvd,Los Angeles,CA,90066,Pest Control,4.0,4,Pest Control,D4ACqt61Ycj6I6ATeU_zaw
9,9,Termigone Exterminators,https://www.yelp.com/biz/termigone-exterminato...,6.262552e+09,780 N Euclid St,Anaheim,CA,92801,Pest Control,4.5,7,Pest Control,kZZ9nR5EHz0K4LJyHi9P2g


In [ ]:
count=0
for zips in tqdm(np.random.choice(zip_codes_remaining,500)):
    try:
        zip_codes_remaining.remove(zips)
    except:
        print(zips)
    #Check if zip code is in a southern state
    if zips in southern_zips:
        for cat in categories:
            count+=1
            print(count)
            try:
                PARAMETERS = {'location':str(zips), 
                          'term': cat,
                          'limit': 50,
                          'offset':50}
                if 5000<=count<=9999:
                    print('API4')
                    HEADERS = {'Authorization': 'bearer %s' % API4}
                if 10000<=count<=14999:
                    print('API3')
                    HEADERS = {'Authorization': 'bearer %s' % API3}
                if 15000<=count<=19999:
                    print('API2')
                    HEADERS = {'Authorization': 'bearer %s' % API2}
                if 20000<=count<=24999:
                    print('API1')
                    HEADERS = {'Authorization': 'bearer %s' % API1}
                if 25000<=count:
                    print('Used up all API requests available')
                    pass
                response = requests.get(url = ENDPOINT, params = PARAMETERS, headers = HEADERS)
                if response.status_code!=200:
                    print('Error: response status code not 200')
                    break
                business_data = response.json()
                for business in business_data['businesses']:
                    name = name + [business['name']]
                    url = url + [business['url']]
                    address = address + [business['location']['address1']]
                    city = city + [business['location']['city']]
                    state = state + [business['location']['state']]
                    czip = czip + [business['location']['zip_code']]
                    phone = phone + [business['phone']]
                    search = search + [cat]
                    yelp_id = yelp_id + [business['id']]
                    industries = str()
                    for i in business['categories']:
                        industries = industries+', '+str(i['title'])
                    vertical = vertical + [industries[2:]]
                    rating = rating + [business['rating']]
                    review_count = review_count + [business['review_count']]
            except:
                pass
    #Else northern state            
    else:
        for cat in ['Air Conditioner & Heating','Electricians','Home Cleaners','Locksmiths','Plumbers']:
            count+=1
            print(count)
            try:
                PARAMETERS = {'location':str(zips), 
                          'term': cat,
                          'limit': 50,
                          'offset':100}
                if 5000<=count<=9999:
                    print('API4')
                    HEADERS = {'Authorization': 'bearer %s' % API4}
                if 10000<=count<=14999:
                    print('API3')
                    HEADERS = {'Authorization': 'bearer %s' % API3}
                if 15000<=count<=19999:
                    print('API2')
                    HEADERS = {'Authorization': 'bearer %s' % API2}
                if 20000<=count<=24999:
                    print('API1')
                    HEADERS = {'Authorization': 'bearer %s' % API1}
                if 25000<=count:
                    print('Used up all API requests available')
                    pass
                response = requests.get(url = ENDPOINT, params = PARAMETERS, headers = HEADERS)
                if response.status_code!=200:
                    print('Error: response status code not 200')
                    break
                business_data = response.json()
                for business in business_data['businesses']:
                    name = name + [business['name']]
                    url = url + [business['url']]
                    address = address + [business['location']['address1']]
                    city = city + [business['location']['city']]
                    state = state + [business['location']['state']]
                    czip = czip + [business['location']['zip_code']]
                    phone = phone + [business['phone']]
                    search = search + [cat]
                    yelp_id = yelp_id + [business['id']]
                    industries = str()
                    for i in business['categories']:
                        industries = industries+', '+str(i['title'])
                    vertical = vertical + [industries[2:]]
                    rating = rating + [business['rating']]
                    review_count = review_count + [business['review_count']]
            except:
                pass